# DIFFERENTIAL EXPRESSION

##### ! featureCounts -a /projects/ps-yeolab/genomes/ce11/c_elegans.PRJNA13758.WS254.annotations.genes.SAF -F SAF -s 2 -o /home/bay001/projects/hundley_rnae_20160210/analysis/rnae_v6/counts.txt *.sorted.rg.bam

In [1]:
# options(bitmapType='cairo')
library("DESeq2")
library("RColorBrewer")
library("ggplot2")
library("pheatmap")
library("RColorBrewer")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges


In [2]:
setwd("/home/bay001/projects/codebase/bfx/rscripts")
out_dir <- "/home/bay001/projects/codebase/bfx/rscripts"

In [5]:
tab <- read.table("counts.txt",header=TRUE,sep="\t",row.names=1)
samples <- tab[,6:ncol(tab)]
colnames(samples) <- gsub(".polyATrim.adapterTrim.rmRep.sorted.rg.bam","",colnames(samples))
colnames(samples) <- gsub("analysis.rnae_v6.","",colnames(samples))
samples <- as.matrix(samples)
colnames(samples)

[1] "s1.bam" "s2.bam" "s3.bam" "s5.bam" "s6.bam" "s7.bam"

In [60]:
colData <- read.table( "conditions.txt", header=TRUE,row.names=1, sep='\t')
# rename(colData, c("factor"="CHOSENFACTOR"))
# ?rename

In [61]:
colData

,condition,factor
s1.bam,edit,wt
s2.bam,noedit,adr2
s3.bam,edit,wt
s5.bam,noedit,adr2
s6.bam,edit,wt
s7.bam,noedit,adr2


In [63]:
condition <- "factor"

In [64]:
colData = rename(colData, c(condition="CHOSENFACTOR"))

dds <- DESeqDataSetFromMatrix(countData=samples, colData=colData, design=~CHOSENFACTOR)
# dds$factors <- relevel(dds$factors, ref="EDIT")
# dds_rlog <- rlogTransformation(dds)

In [65]:
colData

,CHOSENFACTOR,CHOSENFACTOR
s1.bam,edit,wt
s2.bam,noedit,adr2
s3.bam,edit,wt
s5.bam,noedit,adr2
s6.bam,edit,wt
s7.bam,noedit,adr2


In [9]:
# 
png(paste0(out_dir,"pca.png"))
data <- plotPCA(dds_rlog, intgroup=c("factors", "type"), returnData=TRUE)
percentVar <- round(100 * attr(data, "percentVar"))
ggplot(data, aes(PC1, PC2, color=factors, shape=type),
       main = "PCA Plot") +
  geom_point(size=3) +
  scale_shape_manual(values=seq(0,8)) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance"))
dev.off()

png 
  2

In [10]:

sampleDists <- dist(t(assay(dds_rlog)))
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- paste(dds_rlog$condition, dds_rlog$type, sep="-")
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
png(paste0(out_dir,"heatmap.png"))
pheatmap(sampleDistMatrix,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         col=colors,
         main = "Heatmap")
dev.off()

png 
  2

In [11]:
dds <- DESeq(dds)
res <- results(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [12]:
write.table(as.data.frame(res),file=paste0(out_dir,'diffexp.tsv'),sep='\t')

In [15]:
dds$type

[1] WT-1   ADR2-2 WT-3   ADR2-4 WT-6   ADR2-7
Levels: ADR2-2 ADR2-4 ADR2-7 WT-1 WT-3 WT-6